# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Häberle  ->  M. Häberle  |  ['M. Häberle']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
C. Clontz  ->  C. Clontz  |  ['C. Clontz']
P. Smith  ->  P. Smith  |  ['P. Smith']
S. Souza  ->  S. Souza  |  ['S. Souza']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
J. Liu  ->  J. Liu  |  ['J. Liu']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
J. Li  ->  J. Li  |  ['J. Li']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
M. Wells  ->  M. Wells  |  ['M. Wells']
T. Henning  ->  T. Henning  |  ['T. Henning']


H. Beuther  ->  H. Beuther  |  ['H. Beuther']
M. Wells  ->  M. Wells  |  ['M. Wells']
T. Henning  ->  T. Henning  |  ['T. Henning']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
M. Wells  ->  M. Wells  |  ['M. Wells']
Arxiv has 72 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.04903


extracting tarball to tmp_2503.04903...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2503.04903/maintext.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'dispersion_profile_table_los' from 'tmp_2503.04903/dispersion_profile_table_los.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'revision1_dispersion_profile_table_pm' from 'tmp_2503.04903/revision1_dispersion_profile_table_pm.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' fro

...

extracting tarball to tmp_2503.05663...

 done.


H. Beuther  ->  H. Beuther  |  ['H. Beuther']


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure beam_lin_size-DIST_no_dist_outl_nf_flag
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure beam_lin_size-DIST_no_dist_outl_nf_flag as tmp_2503.05663/./Plots/beam_lin_size-DIST_no_dist_outl_nf_flag.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure map_RMS_whole_sample_Jybm_hist_v2
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure map_RMS_whole_sample_Jybm_hist_v2 as tmp_2503.05663/./Plots/map_RMS_

Found 179 bibliographic references in tmp_2503.05663/ALMAGAL_III.bbl.
Issues with the citations
syntax error in line 86: '}' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.04903-b31b1b.svg)](https://arxiv.org/abs/2503.04903) | **oMEGACat. VI. Analysis of the overall kinematics of Omega Centauri in 3D: velocity dispersion, kinematic distance, anisotropy, and energy equipartition**  |
|| <mark>M. Häberle</mark>, et al. -- incl., <mark>N. Neumayer</mark>, <mark>C. Clontz</mark>, <mark>P. Smith</mark>, <mark>S. Souza</mark> |
|*Appeared on*| *2025-03-10*|
|*Comments*| *31 pages, 23 Figures, 4 Tables. Accepted for publication in ApJ. Data products available under: this https URL*|
|**Abstract**|            Omega Centauri ($\omega$ Cen) is the Milky Way's most massive globular cluster and is likely the stripped nucleus of an accreted dwarf galaxy. In this paper, we analyze $\omega$ Cen's kinematics using data from oMEGACat, a comprehensive catalog of $\omega$ Cen's central regions, including 1.4 million proper motion measurements and 300,000 spectroscopic radial velocities. Our velocity dispersion profiles and kinematic maps are consistent with previous work but improve on their resolution, precision, and spatial coverage. The cluster's 3D dispersion is isotropic in the core, with increasing radial anisotropy at larger radii. The 2D kinematic maps show an elongation of the velocity dispersion field comparable to the flattening observed photometrically. We find good agreement between proper motions and line-of-sight velocity dispersion and measure a kinematic distance of 5494$\pm$61 pc, the most precise kinematic distance to $\omega$ Cen available. The subset of data with precise metallicity measurements shows no correlation between metallicity and kinematics, supporting the picture of well-mixed stellar populations within the half-light radius of $\omega$ Cen. Finally, we study the degree of energy equipartition using a large range of stellar masses. We find partial energy equipartition in the center that decreases towards large radii. The spatial dependence of the radial energy equipartition is stronger than the tangential energy equipartition. Our kinematic observations can serve as a new reference for future dynamical modeling efforts that will help to further disentangle the complex mass distribution within $\omega$ Cen.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.05555-b31b1b.svg)](https://arxiv.org/abs/2503.05555) | **ALMAGAL I. The ALMA evolutionary study of high-mass protocluster formation in the Galaxy. Presentation of the survey and early results**  |
|| S. Molinari, et al. -- incl., <mark>H. Beuther</mark>, <mark>M. Wells</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-03-10*|
|*Comments*| *21 pages, 18 figures plus appendices. Astronomy & Astrophysics, accepted*|
|**Abstract**|            Fundamental questions about the physics responsible for fragmenting molecular parsec-scale clumps into cores of ~1000 au are still open, that only a statistically significant investigation with ALMA is able to address: what are the dominant agents that determine the core demographics, mass, and spatial distribution as a function of the physical properties of the hosting clumps, their evolutionary stage and the different Galactic environments in which they reside? To what extent extent is fragmentation driven by clumps dynamics or mass transport in filaments? With ALMAGAL we observed the 1.38 mm continuum and lines toward more than 1000 dense clumps in our Galaxy, with M>500M_sun, surface density > 0.1 g/cm2 and d<7.5 kpc. The ACA and two 12-m array setups were used to deliver a minimum resolution of ~1000 au over the entire sample distance range. The sample covers all evolutionary stages from infrared dark clouds (IRDCs) to HII regions from the tip of the Galactic bar to the outskirts of the Galaxy. The spectral setup includes several molecular lines to trace the multiscale physics and dynamics of gas, notably CH3CN, H2CO, SiO, CH3OH, DCN, HC3N, SO etc. We present an initial overview of the observations and the early science product and results, with a first characterization of the morphological properties of the continuum emission. We use "perimeter-versus-area" and convex hull-versus-area metrics to classify the different morphologies. More extended and morphologically complex shapes are found toward clumps that are relatively more evolved and have higher surface densities.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.05559-b31b1b.svg)](https://arxiv.org/abs/2503.05559) | **ALMAGAL II. The ALMA evolutionary study of high-mass protocluster formation in the Galaxy. ALMA data processing and pipeline**  |
|| Á. Sánchez-Monge, et al. -- incl., <mark>H. Beuther</mark>, <mark>M. Wells</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-03-10*|
|*Comments*| *29 pages, 23 figures, accepted for publication in the Astronomy & Astrophysics journal, abstract adapted from original*|
|**Abstract**|            The ALMAGAL Large Program has observed 1017 high-mass star-forming regions distributed throughout the Galaxy, sampling different evolutionary stages and environmental conditions. In this work, we present the acquisition and processing of the ALMAGAL data. The main goal is to set up a robust pipeline that generates science-ready products, with a good and uniform quality across the whole sample. ALMAGAL observations were performed with the Atacama Large Millimeter/submillimeter Array (ALMA). Each field was observed in three different telescope arrays, being sensitive to spatial scales ranging from 1000 au up to 0.1 pc. The spectral setup allows sensitive imaging of the continuum emission at 219 GHz, and it covers multiple molecular spectral lines observed in four different spectral windows that span about 4 GHz in frequency coverage. We have designed a Python-based processing workflow to calibrate and image these observational data. This ALMAGAL pipeline includes an improved continuum determination, suited for line-rich sources; an automatic self-calibration process that improves the dynamical range of the final images; and the combination of data from different telescope arrays to produce science-ready, fully combined images. The fully combined products have spatial resolutions in the range 800-2000 au, and mass sensitivities in the range 0.02-0.07 Mo. We also present a first analysis of the spectral line information included in the ALMAGAL setup, and its potential for future scientific studies. As an example, specific spectral lines at 1000 au scales resolve the presence of multiple outflows in clusters and will help us to search for disk candidates around massive protostars. Moreover, the broad frequency bands provide information on the chemical richness of the different cluster members, which can be used to study the chemical evolution during the formation process of star clusters.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.05663-b31b1b.svg)](https://arxiv.org/abs/2503.05663) | **ALMAGAL III. Compact source catalog: Fragmentation statistics and physical evolution of the core population**  |
|| A. Coletta, et al. -- incl., <mark>H. Beuther</mark>, <mark>M. Wells</mark> |
|*Appeared on*| *2025-03-10*|
|*Comments*| *35 pages, 63 figures, accepted for publication in the Astronomy & Astrophysics journal, abstract adapted from original*|
|**Abstract**|            The mechanisms behind the fragmentation of high-mass dense clumps into compact star-forming cores are fundamental topics in current astrophysical research. The ALMAGAL survey provides the opportunity to study this process at an unprecedented level of detail and statistical significance, featuring high-angular resolution $1.38$ mm ALMA observations of $1013$ massive dense clumps at various Galactic locations. These clumps cover a wide range of distances, masses, surface densities, and evolutionary stages. Here, we present the catalog of compact sources obtained with the CuTEx algorithm from continuum images of the full ALMAGAL clump sample combining ACA-$7$m and $12$m ALMA arrays, reaching a uniform high median spatial resolution of $\sim1400$ au. We discuss the fragmentation properties and the estimated physical parameters of the core population. The ALMAGAL compact source catalog includes $6348$ cores detected in $844$ clumps ($83\%$ of the total), with a number of cores per clump between $1$ and $49$ (median of $5$). The estimated core diameters are mostly within $\sim800-3000$ au (median of $1700$ au). We obtained core masses from $0.002$ to $345\,\mathrm{M_{\odot}}$. We evaluated the variation in the core mass function (CMF) with evolution as traced by the clump $L/M$, finding a clear, robust shift and change in slope among CMFs within subsamples at different stages. This finding suggests that the CMF shape is not constant throughout the star formation process, but rather it builds (and flattens) with evolution, with higher core masses reached at later stages. We found that all cores within a clump grow in mass on average with evolution, and the number of cores increases with the core masses. Our results favor a clump-fed scenario for high-mass star formation, in which cores form as low-mass seeds, and then gain mass while further fragmentation occurs in the clump.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.04913-b31b1b.svg)](https://arxiv.org/abs/2503.04913) | **The HST-Hyperion Survey: Companion Fraction and Overdensity in a z ~ 2.5 Proto-supercluster**  |
|| F. Giddings, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-03-10*|
|*Comments*| *23 pages, 10 figures, 4 tables. Submitted to A&A. Comments welcome*|
|**Abstract**|            We present a study of the galaxy merger and interaction activity within the Hyperion Proto-supercluster at z~2.5 in an effort to assess the occurrence of galaxy mergers and interactions in contrast to the coeval field and their impact on the build up of stellar mass in high density environments at higher-z. For this work, we utilize data from the Charting Cluster Construction with VUDS and ORELSE Survey (C3VO) along with extensive spectroscopic and photometric datasets available for the COSMOS field, including the HST-Hyperion Survey. To evaluate potential merger and interaction activity, we measure the fraction of galaxies with close kinematic companions ($f_{ckc}$) both within Hyperion and the coeval field by means of a Monte Carlo (MC) methodology developed in this work that probabilistically employs our entire combined spectroscopic and photometric dataset. We validate our $f_{ckc}$ MC methodology on a simulated lightcone built from the GAlaxy Evolution and Assembly semi-analytic model, and we determine correction factors that account for the underlying spectroscopic sampling rate of our dataset. We find that galaxies in Hyperion have close kinematic companions $\gtrsim 2\times$ more than galaxies in the field and measure a corrected $f_{ckc}=49_{-7.8}^{+7.4}$% for Hyperion and a corrected $f_{ckc}=23_{-1.3}^{+1.2}$% for the surrounding field; a $>3\sigma$ difference. This increase in $f_{ckc}$ indicates an enhancement in the merger and interaction activity within Hyperion and matches the trend seen in other structures. The rate of merger and interactions within the field implied from our field $f_{ckc}$ measurement is well aligned with values measured from other observations in similar redshift ranges. The enhanced $f_{ckc}$ measured within Hyperion suggests that merger and interaction activity play an important role in the mass growth of galaxies in denser environments at higher z.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.05198-b31b1b.svg)](https://arxiv.org/abs/2503.05198) | **The JCMT BISTRO Survey: Unveiling the Magnetic Fields around Galactic Center**  |
|| M.-Z. Yang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-03-10*|
|*Comments*| *Accepted for publication in ApJ. 31 pages, 21 figures (20 in main text, 1 in appendix), 1 appendix*|
|**Abstract**|            We acquired 450 {\mu}m and 850 {\mu}m dust continuum polarization observations toward the inner region of the Central Molecular Zone (CMZ) as part of the B-Fields In Star-Forming Region Observations (BISTRO) survey using the POL-2 polarimeter on the James Clerk Maxwell Telescope. These observations encompassed three dense structures: the 20 km s{^{-1}} cloud (20MC), 50 km s{^{-1}} cloud (50MC), and circumnuclear disk (CND). Our aim is to investigate the magnetic field morphology and strength in the inner region of the CMZ using polarized dust continuum and the Davis-Chandrasekhar-Fermi method. The magnetic field morphology is highly ordered in all three dense regions. The plane-of-sky magnetic field strengths are {\sim}1 mG for the 20MC and the 50MC, and {\sim}2 mG for the CND. We compare the energy contributions of turbulence, gravity, and thermal motion with that of the magnetic field using the plasma {\beta}, mass-to-flux ratio, and Alfvén Mach number. The outcomes reveal the magnetic field stands out as the predominant factor within the inner region of the CMZ. The dominance of the magnetic field may explain the low star-forming rate in the CMZ. We further investigate the dust grain alignment efficiency by exploring the relationship between polarization fraction and total intensity. The results suggest that dust grains are well aligned with the magnetic fields.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.05461-b31b1b.svg)](https://arxiv.org/abs/2503.05461) | **CHANG-ES XXXIV: Magnetic Field Structure in Edge-On Galaxies Characterising large-scale magnetic fields in galactic halos**  |
|| M. Stein, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-03-10*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics, section "4. Extragalactic astronomy"*|
|**Abstract**|            Understanding galactic magnetic fields is essential for interpreting feedback processes in galaxies. Despite their importance, the exact structure of these fields, particularly in galactic halos, remains unclear. Accurate descriptions are crucial for understanding the interaction between star formation and halo magnetisation. By systematically analysing the polarisation patterns in halos of nearby galaxies, we aim to deepen the understanding of the interplay between galactic magnetic fields and star formation processes. We provide an analytical description of the observed X-shaped halos. Based on radio polarimetry data, we classify the polarisation patterns of a sample of edge-on galaxies, by using a newly introduced three-class system: disc dominated, small-scale, and X-shaped. We then fit X-shaped patterns to the polarisation data for galaxies classified as X-shaped and explore links between the polarisation patterns and other physical properties of these galaxies. The classification process shows that 11 out of 18 analysed galaxies display an X-shaped polarisation pattern. Galaxies classified as disc dominated seem less efficient at forming stars than expected for their stellar mass and rotate faster than galaxies with similarly sized HI-discs. X-shape modelling reveals that the polarisation patterns are best fitted by a constant-angle model, and we observe a correlation between the X-shape opening angle and star formation rate surface density indicating the interplay between the star formation in the disc and the magnetisation of the galactic halo. The analysis of polarisation patterns in nearby galaxies reveals that most exhibit an X-shaped configuration, indicating a common magnetic field structure in galactic halos. The introduced models capture the X-shaped morphology and reveal the link between the X-shape's opening angle and star formation rate surface density.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.05402-b31b1b.svg)](https://arxiv.org/abs/2503.05402) | **The Chemical Composition of the Sun**  |
|| <mark>M. Bergemann</mark>, K. Lodders, H. Palme |
|*Appeared on*| *2025-03-10*|
|*Comments*| *This is a pre-print of a chapter for the Encyclopedia of Astrophysics (edited by I. Mandel, section editor F.R.N. Schneider) to be published by Elsevier as a Reference Module*|
|**Abstract**|            This chapter provides a brief introduction to the chemical composition of the Sun. The focus of the chapter is on results obtained from the physical analysis of the solar photosphere. Data obtained from meteorites, solar wind and corona measurements, as well as helioseismology, and solar neutrinos are briefly reviewed. The elemental and isotopic composition of the solar system is derived by combining the solar and meteoritic data. The cosmochemical and astronomical abundance scales are described. The results of the determinations of the protosolar chemical composition, as well as the initial and present-day mass fractions of hydrogen, helium, and metals (X, Y, Z) for the solar system are presented in extensive tables. All tables are also available in machine-readable form via Zenodo this https URL |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.04903/./revision1_old_data_velocity_dispersion_map_n250_with_zoom.png', 'tmp_2503.04903/./velocity_dispersion_Gaussian_Fits.png', 'tmp_2503.04903/./revision1_watkins_comparison.png']
copying  tmp_2503.04903/./revision1_old_data_velocity_dispersion_map_n250_with_zoom.png to _build/html/
copying  tmp_2503.04903/./velocity_dispersion_Gaussian_Fits.png to _build/html/
copying  tmp_2503.04903/./revision1_watkins_comparison.png to _build/html/
exported in  _build/html/2503.04903.md
    + _build/html/tmp_2503.04903/./revision1_old_data_velocity_dispersion_map_n250_with_zoom.png
    + _build/html/tmp_2503.04903/./velocity_dispersion_Gaussian_Fits.png
    + _build/html/tmp_2503.04903/./revision1_watkins_comparison.png
found figures ['tmp_2503.05555/./emissionarea-vs-surfdens_tm1_nsig5.png', 'tmp_2503.05555/./almagal_updated_galpos_Final.png', 'tmp_2503.05555/./LM_4paper0a_v2.png', 'tmp_2503.05555/./circ-depart_total_tm1_nsig5-vs-SURF_D.png']
copying  tmp_2503.05555/./em

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\todo}[1]{{\textcolor{red}{TODO: #1}}}$
$\newcommand$
$\newcommand{\revision}[1]{{\textcolor{blue}{\textbf{#1}}}}$
$\newcommand{\sectionautorefname}{Section}$
$\newcommand{\subsectionautorefname}{Section}$
$\newcommand{\subsubsectionautorefname}{Section}$
$\newcommand{\figureautorefname}{Fig.}$</div>



<div id="title">

# oMEGACat. VI. Analysis of the overall kinematics of Omega Centauri in 3D: velocity dispersion, kinematic distance, anisotropy, and energy equipartition

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.04903-b31b1b.svg)](https://arxiv.org/abs/2503.04903)<mark>Appeared on: 2025-03-10</mark> -  _31 pages, 23 Figures, 4 Tables. Accepted for publication in ApJ. Data products available under: this https URL_

</div>
<div id="authors">

<mark>M. Häberle</mark>, et al. -- incl., <mark>N. Neumayer</mark>, <mark>C. Clontz</mark>

</div>
<div id="abstract">

**Abstract:** Omega Centauri ( $\omc$ ) is the Milky Way's most massive globular cluster and is likely the stripped nucleus of an accreted dwarf galaxy. In this paper, we analyze $\omc$ 's kinematics using data from oMEGACat, a comprehensive catalog of $\omc$ 's central regions, including 1.4 million proper motion measurements and 300,000 spectroscopic radial velocities.  Our velocity dispersion profiles and kinematic maps are consistent with previous work but improve on their resolution, precision, and spatial coverage.  The cluster's 3D dispersion is isotropic in the core, with increasing radial anisotropy at larger radii. The 2D kinematic maps show an elongation of the velocity dispersion field comparable to the flattening observed photometrically.We find good agreement between proper motions and line-of-sight velocity dispersion and measure a kinematic distance of $5494\pm61$ pc, the most precise kinematic distance to $\omc$ available.The subset of data with precise metallicity measurements shows no correlation between metallicity and kinematics, supporting the picture of well-mixed stellar populations within the half-light radius of $\omc$ .Finally, we study the degree of energy equipartition using a large range of stellar masses. We find partial energy equipartition in the center that decreases towards large radii. The spatial dependence of the radial energy equipartition is stronger than the tangential energy equipartition.Our kinematic observations can serve as a new reference for future dynamical modeling efforts that will help to further disentangle the complex mass distribution within $\omc$ .

</div>

<div id="div_fig1">

<img src="tmp_2503.04903/./revision1_old_data_velocity_dispersion_map_n250_with_zoom.png" alt="Fig9" width="100%"/>

**Figure 9. -** Dispersion map combining both proper motion components and determined using $N=250$ Voronoi bins. The right panel shows a zoom into the centermost arcminute with the numerical values (in mas yr$^{-1}$) for the individual bins shown in black letters. (*fig:voronoi_map_proper_motion*)

</div>
<div id="div_fig2">

<img src="tmp_2503.04903/./velocity_dispersion_Gaussian_Fits.png" alt="Fig10" width="100%"/>

**Figure 10. -** _Top:_ Result of a single component 2D Gaussian fit to the 2D proper motion dispersion field shown from \autoref{fig:voronoi_map_proper_motion}. The _left_ shows the result, the _center_ shows the residuals, which are in good agreement besides an underestimation of the cusp of the velocity dispersion in the centermost region, and the _right_ shows the parameters of the Gaussian fit. _Bottom:_ Results for a 2-component 2D Gaussian fit. This 2-component model is better able to describe the velocity dispersion in the innermost region, as can be seen from the residuals. Both models successfully recover the ellipticity and the position angle of the dispersion field. (*fig:voronoi_map_gaussian_fit*)

</div>
<div id="div_fig3">

<img src="tmp_2503.04903/./revision1_watkins_comparison.png" alt="Fig2" width="100%"/>

**Figure 2. -** Proper motion dispersion profiles determined using the new oMEGACat (black markers). We compare the new dispersion measurements with the literature profile by \citetalias{2015ApJ...803...29W}(blue markers). For better comparability, in the _top panel_ we restrict our data set to bright stars and use a binning scheme similar to the literature. In the _center panel_ we use logarithmic radial bins and the full high-quality subset. In the _bottom panel_ we compare the anisotropy, again using the bright sample only. (*fig:watkins_comparison*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.04903"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\lbol}{L_{\mathrm{bol}}}$
$\newcommand{\um}{\mum}$
$\newcommand{\adeg}{^{\circ}}$
$\newcommand{\hi}{H{\sc i}}$
$\newcommand{\hii}{H{\sc ii}}$
$\newcommand{\chiiicn}{CH_3CN (12--11)}$
$\newcommand{\chiiitcn}{CH_3^{13}CN (12--11)}$
$\newcommand{\hiico}{H_2CO}$
$\newcommand{\chiiioh}{CH_3OH}$
$\newcommand{\dcn}{DCN (3 -- 2)}$
$\newcommand{\ocs}{OCS (19 -- 18)}$
$\newcommand{\hciiin}{HC_3N(24 -- 23)}$
$\newcommand{\so}{SO (6_5 -- 5_4)}$
$\newcommand{\sio}{SiO (5 -- 4)}$
$\newcommand{\tco}{^{13}CO (2--1)}$
$\newcommand{\ceo}{C^{18}O (2--1)}$
$\newcommand{\vlsr}{v_{\rm LSR}}$
$\newcommand{\gcmtwo}{g cm^{-2}}$
$\newcommand{\asec}{^{\prime \prime}}$
$\newcommand{\deltacirc}{\Delta^{\rm circ} _{5\sigma}}$
$\newcommand{\qhull}{Q^{\rm hull} _{5\sigma}}$
$\newcommand{\fres}{\textit{\sc 7m+tm2+tm1}}$
$\newcommand{\ires}{\textit{\sc 7m+tm2}}$
$\newcommand{\lres}{\textit{\sc 7m}}$
$\newcommand{\n}{"near"}$
$\newcommand{\f}{"far"}$
$\newcommand{\cpear}{r_P}$
$\newcommand{\revision}[1]{\textbf{\color{red} #1}}$
$\newcommand{\nhigal}{915}$
$\newcommand{\nrms}{98}$
$\newcommand{\nall}{1013}$
$\newcommand{\nvlim}{77}$
$\newcommand{\ndnf}{24}$
$\newcommand{\ndfn}{39}$
$\newcommand{\ndfiveh}{95}$
$\newcommand{\nkdanewcommandnear}{7}$
$\newcommand{\nnewcommandnear}{7}$
$\newcommand{\samplen}{538}$
$\newcommand{\samplef}{479}$</div>



<div id="title">

# ALMAGAL I. The ALMA evolutionary study of high-mass protocluster formation in the Galaxy: Presentation of the survey and early results

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.05555-b31b1b.svg)](https://arxiv.org/abs/2503.05555)<mark>Appeared on: 2025-03-10</mark> -  _21 pages, 18 figures plus appendices. Astronomy & Astrophysics, accepted_

</div>
<div id="authors">

S. Molinari, et al. -- incl., <mark>H. Beuther</mark>, <mark>M. Wells</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** A large fraction of stars form in clusters containing high-mass stars, which subsequently influences the local and galaxy-wide environment. Fundamental questions about the physics responsible for fragmenting molecular parsec-scale clumps into cores of a few thousand astronomical units (au)  are still open, that only a statistically significant investigation with ALMA is able to address; for instance: the identification of the dominant agents  that determine the core demographics, mass, and spatial distribution as a function of the physical properties of the hosting clumps, their evolutionary stage and the different Galactic environments in which they reside. The extent to which fragmentation is driven by clumps dynamics or mass transport in filaments also remains elusive. With the ALMAGAL project, we observed the 1.38 mm continuum and lines toward more than 1000 dense clumps in our Galaxy, with $M \geq$ 500 $\msun$ , $\Sigma\geq 0.1$ $\gcmtwo$ and $d$ $\leq$ 7.5 kiloparsec (kpc). Two different combinations of ALMA Compact Array (ACA) and 12-m array setups were used to deliver a minimum resolution of $\sim$ 1000 au over the entire sample distance range. The sample covers all evolutionary stages from infrared dark clouds (IRDCs) to $\hii$ regions from the tip of the Galactic bar  to the outskirts of the Galaxy. With a continuum sensitivity of 0.1 mJy, ALMAGAL enables a complete study of the clump-to-core fragmentation process down to $M\sim 0.3 $ ${\msun}$ across the Galaxy. The spectral setup includes several molecular lines to trace the multiscale physics and dynamics of gas, notably $CH_3$ CN, $H_2$ CO, SiO, $CH_3$ OH, DCN, $HC_3$ N, and SO, among others. We present an initial overview of the observations and the early science product and results produced in the ALMAGAL Consortium, with a first characterization of the morphological properties of the continuum emission detected above 5 $\sigma$ in our fields. We used "perimeter-versus-area" and convex hull-versus-area metrics to classify the different morphologies. We find that more extended and morphologically complex (significantly departing from circular or generally convex) shapes are found toward clumps that are relatively more evolved and have higher surface densities. ALMAGAL is poised to  serve as a game-changer for a number of specific issues in star formation: clump-to-core fragmentation processes, demographics of cores, core and clump gas chemistry and dynamics, infall and outflow dynamics, and disk detections. Many of these issues will  be covered in the first generation of papers that closely follow on the present publication.

</div>

<div id="div_fig1">

<img src="tmp_2503.05555/./emissionarea-vs-surfdens_tm1_nsig5.png" alt="Fig5" width="100%"/>

**Figure 5. -** ALMA 1.38 mm total emission area above 5$\sigma$ in $\fres$ images as a function of the clump surface density from Table 1. The green symbols are for target clumps with $L/M\geq$5. The red histogram reports the fractional distribution of the surface density  for ALMAGAL sources with no 5$\sigma$ detection (to be read on the red right $y$-axis). (*roi-area_surf_d*)

</div>
<div id="div_fig2">

<img src="tmp_2503.05555/./almagal_updated_galpos_Final.png" alt="Fig20.1" width="50%"/><img src="tmp_2503.05555/./LM_4paper0a_v2.png" alt="Fig20.2" width="50%"/>

**Figure 20. -** (a) Galactic distribution of ALMAGAL target clumps are shown on the left, with a symbol size of $\propto$Log($M_c$) (clump mass) and color coded by $\Sigma_c$(surface density), as determined from the Hi-GAL data (non-beam-deconvolved), reflecting the updated distances and physical parameters described in Sect. \ref{revision} and reported in Table 1, available at the CDS. Dotted circles represent 1 kpc distance interval centered on the Sun and the plus symbol marks the location of the Galactic Center. The blue dashed circle marks the distance threshold used to split the target sample among the two ALMA antenna configurations designed to provide a minimum 1000 au linear resolution. The red circle marks the 7.5 kpc distance originally used as an upper limit for source selection. b)  L$_{bol}$/M$_{clump}$ plot for the 1017 selected clumps (color-coded by T$_{dust}$) is shown on the right.
  Asterisks indicate Hi-GAL sample sources associated with $\hi$i regions from the surveys of CORNISH  ([Purcell, Hoare and Cotton 2013]()) , and CORNISH-S  ([Irabor, Hoare and Burton 2023]()) .
   (*sample_fig*)

</div>
<div id="div_fig3">

<img src="tmp_2503.05555/./circ-depart_total_tm1_nsig5-vs-SURF_D.png" alt="Fig9" width="100%"/>

**Figure 9. -** Departure from circularity, $\deltacirc$, of the total of the 5$\sigma$ ALMA emission RoIs in each field as a function of the clump surface density. The red asterisks are the medians of $\deltacirc$ in logarithmic bins of surface density. (*deltacirc_surfd_fig*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.05555"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\phn}{\phantom{0}}$
$\newcommand{\phs}{\phantom{<}}$
$\newcommand{\hii}{\ion{H}{ii}}$</div>



<div id="title">

# ALMAGAL II. The ALMA evolutionary study of high-mass protocluster formation in the Galaxy: ALMA data processing and pipeline

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.05559-b31b1b.svg)](https://arxiv.org/abs/2503.05559)<mark>Appeared on: 2025-03-10</mark> -  _29 pages, 23 figures, accepted for publication in the Astronomy & Astrophysics journal, abstract adapted from original_

</div>
<div id="authors">

Á. Sánchez-Monge, et al. -- incl., <mark>H. Beuther</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Stars form preferentially in clusters embedded inside massive molecular clouds, many of which contain high-mass stars. Thus, a comprehensive understanding of star formation requires a robust and statistically well-constrained characterization of the formation and early evolution of these high-mass star clusters. To achieve this, we designed the ALMAGAL Large Program that observed 1017 high-mass star-forming regions distributed throughout the Galaxy, sampling different evolutionary stages and environmental conditions. In this work, we present the acquisition and processing of the ALMAGAL data. The main goal is to set up a robust pipeline that generates science-ready products, that is, continuum and spectral cubes for each ALMAGAL field, with a good and uniform quality across the whole sample. ALMAGAL observations were performed with the Atacama Large Millimeter/submillimeter Array (ALMA). Each field was observed in three different telescope arrays, being sensitive to spatial scales ranging from $\approx1000$ au up to $\approx0.1$ pc. The spectral setup allows sensitive ( $\approx0.1$ mJy beam $^{-1}$ ) imaging of the continuum emission at 219 GHz (or 1.38 mm), and it covers multiple molecular spectral lines observed in four different spectral windows that span about $\approx4$ GHz in frequency coverage. We have designed a Python-based processing workflow to calibrate and image these observational data. This ALMAGAL pipeline includes an improved continuum determination, suited for line-rich sources; an automatic self-calibration process that reduces phase-noise fluctuations and improves the dynamical range by up to a factor $\approx5$ in about 15 \% of the fields; and the combination of data from different telescope arrays to produce science-ready, fully combined images. The final products are a set of uniformly generated continuum images and spectral cubes for each ALMAGAL field, including individual-array and combined-array products.The fully combined products have spatial resolutions in the range 800--2000 au, and mass sensitivities in the range 0.02--0.07 $M_\odot$ . We also present a first analysis of the spectral line information included in the ALMAGAL setup, and its potential for future scientific studies. As an example, specific spectral lines (e.g., SiO and $CH_3$ CN) at $\approx1000$ au scales resolve the presence of multiple outflows in clusters and will help us to search for disk candidates around massive protostars. Moreover, the broad frequency bands provide information on the chemical richness of the different cluster members, which can be used to study the chemical evolution during the formation process of star clusters.

</div>

<div id="div_fig1">

<img src="tmp_2503.05559/./plot_jointdeconvolutionExamples_101899_noGrid.png" alt="Fig15" width="100%"/>

**Figure 15. -** Joint-deconvolution imaging strategy used in the ALMAGAL processing workflow. The target used as an example is the ALMAGAL field 101899 (also labelled AG023.0108$-$0.4102 in \citealt{Molinari2024}). The different columns show from left to right: CLEANed image, residual image, CLEANing mask, and histogram of residual emission. The different rows show the four main steps of the iterative CLEANing process (see details in Sect. \ref{sec:joint-deconvolution}). Note that in Step 1, there is no starting CLEANing mask and the output (shown in the panel) is generated from the multi-thresh automasking functionalities of \texttt{tclean}. (*fig:imaging-strategy*)

</div>
<div id="div_fig2">

<img src="tmp_2503.05559/./plot_beamInformation_noGrid.png" alt="Fig20" width="100%"/>

**Figure 20. -** Distribution of the beam properties for the ALMAGAL images. The $y$-axis in all panels depicts the number of images. From top to bottom, each row corresponds to the images for the 7M, TM2, TM1, 7M+TM2, and 7M+TM2+TM1 array configurations. The columns show from left to right, the beam major axis ($\theta_\mathrm{maj}$), the beam minor axis ($\theta_\mathrm{min}$), and the beam elongation ($\theta_\mathrm{maj}/\theta_\mathrm{min}$). Light and dark histograms correspond to the "near" and "far" samples, respectively. (*fig:beam-properties*)

</div>
<div id="div_fig3">

<img src="tmp_2503.05559/./plot_rmsContInformation_noGrid.png" alt="Fig21" width="100%"/>

**Figure 21. -** Distribution of the noise level (left column), peak intensity (central column), and dynamic range (right column) for the continuum ALMAGAL images. The $y$-axis in all panels depicts the number of images. From top to bottom, each row corresponds to the images for the 7M, TM2, TM1, 7M+TM2, and 7M+TM2+TM1 array configurations. The dynamic range (or signal-to-noise ratio) is determined as the ratio between the peak intensity and the noise level, with the noise level derived as the median absolute deviation (MAD) of the residual image (see Sect. \ref{sec:QAintensity}). (*fig:continuum-properties*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.05559"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\juan}[1]{{\bf #1}}$</div>



<div id="title">

# ALMAGAL III. Compact source catalog: Fragmentation statistics and physical evolution of the core population

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.05663-b31b1b.svg)](https://arxiv.org/abs/2503.05663)<mark>Appeared on: 2025-03-10</mark> -  _35 pages, 63 figures, accepted for publication in the Astronomy & Astrophysics journal, abstract adapted from original_

</div>
<div id="authors">

A. Coletta, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** The physical mechanisms behind the fragmentation of high-mass dense clumps into compact star-forming cores and the properties of these cores are fundamental topics that are heavily investigated in current astrophysical research. The ALMAGAL survey provides the opportunity to study this process at an unprecedented level of detail and statistical significance, featuring high-angular resolution $1.38$ mm ALMA observations of $1013$ massive dense clumps at various Galactic locations. These clumps cover a wide range of distances ( $\sim2-8$ kpc), masses ( $\sim10^{2}-10^{4} \mathrm{M_{\odot}}$ ), surface densities ( $0.1-10$ g $\mathrm{cm^{-2}}$ ), and evolutionary stages (luminosity over mass ratio indicator of $\sim0.05<L/M<450 \mathrm{L_{\odot}/M_{\odot}}$ ).Here, we present the catalog of compact sources obtained with the _CuTEx_ algorithm from continuum images of the full ALMAGAL clump sample combining ACA- $7$ m and $12$ m ALMA arrays, reaching a uniform high median spatial resolution of $\sim1400$ au (down to $\sim800$ au). We characterize and discuss the revealed fragmentation properties and the photometric and estimated physical parameters of the core population.The ALMAGAL compact source catalog   includes $6348$ cores detected in $844$ clumps ( $83\%$ of the total), with a number of cores per clump between $1$ and $49$ (median of $5$ ). The estimated core diameters are mostly within $\sim800-3000$ au (median of $1700$ au). We assigned core temperatures based on the $L/M$ of the hosting clump, and obtained core masses from $0.002$ to $345 \mathrm{M_{\odot}}$ (complete above $0.23 \mathrm{M_{\odot}}$ ), exhibiting a good correlation with the core radii ( $M\propto R^{2.6}$ ).We evaluated the variation in the core mass function (CMF) with evolution as traced by the clump $L/M$ , finding a clear, robust shift and change in slope among CMFs within subsamples at different stages. This finding suggests that the CMF shape is not constant throughout the star formation process, but rather it builds (and flattens) with evolution, with higher core masses reached at later stages.We found that all cores within a clump grow in mass on average with evolution, while a population of possibly newly formed lower-mass cores is present throughout. The number of cores increases with the core masses, at least until the most massive core reaches $\sim10 \mathrm{M_{\odot}}$ .More generally, our results favor a clump-fed scenario for high-mass star formation, in which cores form as low-mass seeds, and then gain mass while further fragmentation occurs in the clump.

</div>

<div id="div_fig1">

<img src="tmp_2503.05663/./Plots/CMF_sim_overplot_cumulative_test_new_LM_comp_excl_ff_compl_v2.png" alt="Fig7" width="100%"/>

**Figure 7. -** Observed CMFs (black lines, expressed as inverse cumulative number density distributions) for the three populations of ALMAGAL cores classified, as reported, according to the $L/M$ evolutionary indicator of the hosting clump, as explained in Sect. \ref{Tcore_model}. The three colored bands trace, for each population, the $1000$ MC realizations of CMFs computed by assigning to each core a random temperature within the ranges defined in Table \ref{Tcore_tab}. Less evolved sources (i.e., with $L/M\leq1  \mathrm{L_{\odot}/M_{\odot}}$) are marked by golden lines, the intermediate ones ($1<L/M\leq10  \mathrm{L_{\odot}/M_{\odot}}$) by green lines, and the more evolved sources ($L/M>10 \mathrm{L_{\odot}/M_{\odot}}$) by blue lines. Note: we include only cores above the mass completeness value of $0.23 \mathrm{M_{\odot}}$, marked by the vertical dashed magenta line (see text for explanation). (*CMF_evolution*)

</div>
<div id="div_fig2">

<img src="tmp_2503.05663/./Plots/Mcore_max+min-L_M_Merr_v4.png" alt="Fig25.1" width="50%"/><img src="tmp_2503.05663/./Plots/Mcore_max-Total_Mcore-Mcore_max_ccL_M_v5.png" alt="Fig25.2" width="50%"/>

**Figure 25. -** Evolution of core masses. _Left panel_: Median values of the maximum ($M_{\rm{core}}^{\rm{max}}$, red) and the minimum ($M_{\rm{core}}^{\rm{min}}$, blue) core masses revealed within different bins of clump $L/M$(delimited by vertical dashed grey lines). The respective colored shaded areas mark the 25th and 75th percentiles of the two distributions within the same $L/M$ bins.
    _Right panel_: Combined mass of the cores ($M_{\rm{core}}^{\rm{tot}}$) minus the mass of the MMC within each clump, as a function of the latter. Data points are color-coded by the clump $L/M$. The solid black line draws the trend of the median value of the quantity on the $y$-axis in given bins of $M_{\rm{core}}^{\rm{max}}$. The dashed grey line corresponds to the $M_{\rm{core}}^{\rm{max}}=50\% M_{\rm{core}}^{\rm{tot}}$ relation, while the shaded grey area marks (from top to bottom) the $25-75\%$ range. (*core_mass_growth*)

</div>
<div id="div_fig3">

<img src="tmp_2503.05663/./Plots/beam_circ_near-far_hist_norm.png" alt="Fig14.1" width="50%"/><img src="tmp_2503.05663/./Plots/beam_circ_lin_size_hist_overplot.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** Beam properties of the ALMAGAL continuum maps. _Left panel_: Probability density distribution of the circularized angular sizes of the map beam, separately for near (red) and far (blue) observational samples, respectively (see \citealt{Sanchez+25} for more details).
    _Right panel_: Distribution of the corresponding linear sizes (i.e., achieved spatial resolution). The grey histogram represents the overall distribution, with the vertical dashed black line marking its median value ($1400$ au). Near and far subsamples are overplotted with the same color coding of the left panel.
    It can be noted that, due to the distance variation across our sample, the bimodal distribution in the left panel converts into a fairly uniform beam linear size overall distribution (see text for details). (*beam_circ_props_plots*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.05663"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

356  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
